In [1]:
#FIUBA - 75.26 Simulacion
#2C 2019 - Trabajo final
#79979 - Gonzalez, Juan Manuel (juanmg0511@gmail.com)

#A pseudo-random numbers generator based on a novel 3D chaotic map with an application to color image encryption
#https://doi.org/10.1007/s11071-018-4390-z
#A color image encryption scheme

#Importación modulos y librerias
import sys, os, math
from PIL import Image

#SECCION DE CONFIGURACION
#Path del archivo original
path_original = "images/lena_std.tif"
#Initial guess
xd0 = 0.411
yd0 = 0.321
zd0 = 0.631
#FIN SECCION DE CONFIGURACION

#Determinación de path con nombre y extensión
path_original_nombre, path_original_extension = os.path.splitext(path_original)

path_encriptada = path_original_nombre
path_desencriptada = path_original_nombre
if (path_original_extension == ""):
    path_encriptada += "_encrypted"
    path_desencriptada += "_decrypted"
else:
    path_encriptada += "_encrypted" + path_original_extension
    path_desencriptada += "_decrypted" + path_original_extension

#Definición de parámetros de control: c1, c2, reales
c1 = 20
c2 = 20
#Cantidad de iteraciones y precisión
nt = 10**5
d = 8

#Implementación de funciones auxiliares
#Picewise map
def pMap(x):
    "piecewise map: funcion que dado x y un parametro de control real c1, calcula Ψ(x)"
    return (abs(1 - (c1 * x)))

#Logistic map
def lMap(x, y):
    "2D logistic map: funcion que dados x e y, y un parametro de control real c2, calcula Λ(x, y)"
    return (c2 * x * (1 - y))

#Implemantación del 3D piecewise-logistic map (3D-PLM), T
def tMap(x, y, z):
    "3D piecewise-logistic map: funcion que dados x, y, z, los paráetros de control reales c1 y c2, calcula T(x, y, z)"    
    x = (pMap(x) + lMap(y, z)) % 1
    y = (pMap(y) + lMap(z, x)) % 1
    z = (pMap(z) + lMap(x, y)) % 1
    
    return x, y, z

#Función de discretización Φ
def phi(amin, amax, d, u):
    "Función de discretización Φ:"
    "Φd : [amin, amax] -> [0, ... , d] ; para amin=0 y amax=1"
    "Dados amin, amax y d, calcula: u -> Φd(u)"
    
    h = ((abs(amax-amin))/(d+1))
    
    r = 0
    if (u >= amin and u < amax):   
        r = math.floor((u-amin)/h)    
    else:
        r = d

    return (r)

#Función para generar las mascaras caóticas aplicadas por el algoritmo de encriptación
def generateChaoticMasks(K, m, n, nt):
    "Genera las mascaras caoticas utilizando el 3D-PLM"
    "Entrada:"
    "K: la clave de encriptacion"
    "m,n: las dimensiones de la imagen, alto y ancho respectivamente"
    "nt: parametro de transicion del 3D-PLM"
    "Salida:"
    "Las mascaras Mr, Mg, Mb"
    
    Cx = []
    Cy = []
    Cz = []

    t = TicToc()
    t.tic()
    for i in range(m*n):
        x = K[0]
        y = K[1]
        z = K[2]
        for j in range(nt+i):
            x, y, z = tMap(x, y, z)
        Cx.append(x)
        Cy.append(y)
        Cz.append(z)
        barraProgreso("Generating the chaotic masks",i,(m*n))
        t.toc()
    barraProgreso("Done",(m*n),(m*n))
    t.toc()

    Mr = []
    Mg = []
    Mb = []

    for i in range(m*n):
        Mr.append(phi(0,1,(m*n),Cx[i]))
        Mg.append(phi(0,1,(m*n),Cy[i]))
        Mb.append(phi(0,1,(m*n),Cz[i]))

    return Mr, Mg, Mb

#Función para generar las mascaras caóticas aplicadas por el algoritmo de encriptación
#Versión optimizada
def generateChaoticMasksV2(K, m, n, nt):
    "Genera las mascaras caoticas utilizando el 3D-PLM"
    "Entrada:"
    "K: la clave de encriptacion"
    "m,n: las dimensiones de la imagen, alto y ancho respectivamente"
    "nt: parametro de transicion del 3D-PLM"
    "Salida:"
    "Las mascaras Mr, Mg, Mb"

    Cx = []
    Cy = []
    Cz = []

    x = K[0]
    y = K[1]
    z = K[2]
    for i in range(nt):
        x, y, z = tMap(x, y, z)
    
    #t = TicToc()
    #t.tic()
    for i in range(m*n):
        if (i==0):
            Cx.append(x)
            Cy.append(y)
            Cz.append(z)
        else:
            x, y, z = tMap(Cx[i-1], Cy[i-1], Cz[i-1])
            Cx.append(x)
            Cy.append(y)
            Cz.append(z)
        #barraProgreso("Generating the chaotic masks",i,(m*n))
        #t.toc()
    #barraProgreso("Done",(m*n),(m*n))
    #t.toc()

    Mr = []
    Mg = []
    Mb = []

    for i in range(m*n):
        Mr.append(phi(0,1,(m*n),Cx[i]))
        Mg.append(phi(0,1,(m*n),Cy[i]))
        Mb.append(phi(0,1,(m*n),Cz[i]))

    return Mr, Mg, Mb

In [2]:
#Cargamos los valores de las variables
#Ejecutar solamente para hacer pruebas, ya están generadas las máscaras caóticas para el archivo 'lena_std.tif'
#dill.load_session("encription.scheme.db")

In [3]:
#Apertura del archivo de imagen
im_original = Image.open(path_original)
Io = list(im_original.getdata())

n = im_original.width
m = im_original.height

In [4]:
#Encriptación
#Paso 1
#Preliminary steps
Ir = [] 
Ig = []
Ib = []

Ir = [r[0] for r in Io]
Ig = [g[1] for g in Io]
Ib = [b[2] for b in Io]

In [5]:
#Paso 2
#Generation of the initial conditions
Er = sum(Ir)
Eg = sum(Ig)
Eb = sum(Ib)

x0 = xd0 + (10**(-(d+8)) * Er)
y0 = yd0 + (10**(-(d+8)) * Eg)
z0 = zd0 + (10**(-(d+8)) * Eb)

K = (x0, y0, z0)

In [6]:
#Paso 3
#Obtain the shuffled image
x = K[0]
y = K[1]
z = K[2]

kr = 0
kg = 0
kb = 0

Sr = []
Sg = []
Sb = []

i = 0
for i in range(n*m):
    Sr.append(-1)
    Sg.append(-1)
    Sb.append(-1)

while (kr < n*m or kg < n*m or kb < n*m):

    x, y, z = tMap(x, y, z)

    if ((Sr[phi(0,1,(m*n)-1,x)]) < 0):
        Sr[phi(0,1,(m*n)-1,x)] = Ir[kr]
        kr = kr + 1
    
    if ((Sg[phi(0,1,(m*n)-1,y)]) < 0):
        Sg[phi(0,1,(m*n)-1,y)] = Ig[kg]
        kg = kg + 1
        
    if ((Sb[phi(0,1,(m*n)-1,z)]) < 0):
        Sb[phi(0,1,(m*n)-1,z)] = Ib[kb]
        kb = kb + 1

In [7]:
#Paso 4
#Generate the chaotic masks

Mr, Mg, Mb = generateChaoticMasksV2(K,m,n,nt)

In [8]:
#Paso 5
#Produce the cipher image
Cpr = []
Cpg = []
Cpb = []

for i in range(m*n):
    Cpr.append((Sr[i] + Mr[i]) % 256)
    Cpg.append((Sg[i] + Mg[i]) % 256)
    Cpb.append((Sb[i] + Mb[i]) % 256)

Ic = []
for i in range(m*n):
    Ic.append((Cpr[i], Cpg[i], Cpb[i]))

im_encriptada = Image.new(im_original.mode, im_original.size)
im_encriptada.putdata(Ic)
im_encriptada.save(path_encriptada,format=im_original.format,quality=100,subsampling=0)
print("Imagen encriptada con clave K:")
print(K)

Imagen encriptada con clave K:
(0.4110000047244551, 0.3210000025965682, 0.6310000027632665)


In [9]:
#Verificamos el resultado de la encriptación
#Imagen original
im_original.show()
#Imagen encriptada
im_encriptada.show()

In [10]:
#Des-encriptación
#Apertura de la imagen
im_encriptada_sc = Image.open(path_encriptada)
Ie = list(im_encriptada_sc.getdata())

n = im_encriptada_sc.width
m = im_encriptada_sc.height

#Paso 1
#Preliminary stage
Cpr = []
Cpg = []
Cpb = []

Cpr = [r[0] for r in Ie]
Cpg = [g[1] for g in Ie]
Cpb = [b[2] for b in Ie]

In [11]:
#Paso 2
#Generate the chaotic masks

#Ejecutar este paso es opcional
#Las mascaras son las mismas tanto para encriptar o desencriptar
#Mr, Mg, Mb = generateChaoticMasksV2(K,m,n,nt)

In [12]:
#Paso 3
#Obtain the shuffled image
Sr = []
Sg = []
Sb = []

for i in range(m*n):
    Sr.append((Cpr[i] - Mr[i]) % 256)
    Sg.append((Cpg[i] - Mg[i]) % 256)
    Sb.append((Cpb[i] - Mb[i]) % 256)

In [13]:
#Paso 4
#Retrieve the original image
x = K[0]
y = K[1]
z = K[2]

kr = 0
kg = 0
kb = 0

Ir = []
Ig = []
Ib = []

while (kr < n*m or kg < n*m or kb < n*m):

    x, y, z = tMap(x, y, z)

    if ((Sr[phi(0,1,(m*n)-1,x)]) >= 0):
        Ir.append(Sr[phi(0,1,(m*n)-1,x)])
        Sr[phi(0,1,(m*n)-1,x)] = -1
        kr = kr + 1
    
    if ((Sg[phi(0,1,(m*n)-1,y)]) >= 0):
        Ig.append(Sg[phi(0,1,(m*n)-1,y)])
        Sg[phi(0,1,(m*n)-1,y)] = -1
        kg = kg + 1
        
    if ((Sb[phi(0,1,(m*n)-1,z)]) >= 0):
        Ib.append(Sb[phi(0,1,(m*n)-1,z)])
        Sb[phi(0,1,(m*n)-1,z)] = -1
        kb = kb + 1
        
Id = []
for i in range(m*n):
    Id.append((Ir[i], Ig[i], Ib[i]))

im_desencriptada = Image.new(im_encriptada_sc.mode, im_encriptada_sc.size)
im_desencriptada.putdata(Id)
im_desencriptada.save(path_desencriptada,format=im_encriptada_sc.format,quality=100,subsampling=0)

In [14]:
#Verificamos el resultado de la des-encriptación
#Imagen encriptada
im_encriptada_sc.show()
#Imagen des-encriptada
im_desencriptada.show()

In [15]:
#Comparamos las dos imagenes, la original y la des-encriptada
im_original = Image.open(path_original)
im_desencriptada = Image.open(path_desencriptada)

Io = list(im_original.getdata())
Id = list(im_desencriptada.getdata())

n = im_original.width
m = im_original.height

print("Comparando '" + path_original + "' y '" + path_desencriptada + "':")
distintas = True
for i in range(n*n):
    if(Io[i] == Id[i]):
        distintas = False
    else:
        distintas = True

if (distintas == False):
    print("Las imágenes son iguales :)")
else:
    print("Las imágenes son distintas :(")

Comparando 'images/lena_std.tif' y 'images/lena_std_decrypted.tif':
Las imágenes son iguales :)


In [16]:
#Guardamos los valores de las variables
#Guardar solamente para hacer pruebas con una imagen en particular
#dill.dump_session("encription.scheme.db")